In [1]:
import pandas as pd
import json

# Load review data
with open('yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    review_data = [json.loads(line) for line in f]
review_df = pd.DataFrame(review_data)

# Filter relevant columns for baseline model
baseline_data = review_df[['user_id', 'business_id', 'stars']]

# Show sample data
print(baseline_data.head())


                  user_id             business_id  stars
0  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw    3.0
1  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ    5.0
2  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A    3.0
3  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA    5.0
4  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ    4.0


In [19]:
from sklearn.model_selection import train_test_split

# # Split the data into 80% training and 20% test sets
# train_data, test_data = train_test_split(baseline_data, test_size=0.2, random_state=42)

# print(f"Training data size: {len(train_data)}")
# print(f"Test data size: {len(test_data)}")

sampled_data = baseline_data.sample(n=12000, random_state=42)
train_data = sampled_data[:10000]
test_data = sampled_data[10000:]

In [17]:
from surprise import KNNBasic
import numpy as np

class JaccardKNN(KNNBasic):
    def compute_similarities(self):
        """Override default similarity computation with Jaccard Similarity."""
        n_x, n_y = self.trainset.n_users, self.trainset.n_users
        sim = np.zeros((n_x, n_y))
        
        # Compute Jaccard similarity
        for i in range(n_x):
            for j in range(n_y):
                u_i = set([item for (item, _) in self.trainset.ur[i]])
                u_j = set([item for (item, _) in self.trainset.ur[j]])
                sim[i][j] = len(u_i & u_j) / len(u_i | u_j) if len(u_i | u_j) != 0 else 0
        return sim


In [ ]:
from surprise import Dataset, Reader, accuracy

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_data[['user_id', 'business_id', 'stars']], reader)
trainset = data.build_full_trainset()

# Use Jaccard-based KNN
model = JaccardKNN(k=20, sim_options={'user_based': True})
model.fit(trainset)

# Create test set
testset = list(test_data.itertuples(index=False, name=None))

# Predict ratings for test set
predictions = model.test(testset)

# Evaluate the model
print("Jaccard Similarity Performance:")
accuracy.rmse(predictions)
accuracy.mae(predictions)

Jaccard Similarity Performance:
RMSE: 1.4751
MAE:  1.2513


1.2512640000000002

In [ ]:
from surprise import Dataset, Reader, accuracy

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_data[['user_id', 'business_id', 'stars']], reader)
trainset = data.build_full_trainset()

# Use Jaccard-based KNN
model = JaccardKNN(k=20, sim_options={'user_based': True})
model.fit(trainset)

# Create test set
testset = list(test_data.itertuples(index=False, name=None))

# Predict ratings for test set
predictions = model.test(testset)

# Evaluate the model
print("Jaccard Similarity Performance:")
accuracy.rmse(predictions)
accuracy.mae(predictions)



Jaccard Similarity Performance:
RMSE: 1.5092
MAE:  1.2730


1.2730377777777777